<a href="https://colab.research.google.com/github/Aaryan-khairnar/AUTOGRADER/blob/main/AutoGrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import getpass
import google.generativeai as genai

api_key = getpass.getpass("Enter your Gemini API Key: ")
genai.configure(api_key=api_key)

In [ ]:
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
}

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-1.5-flash",
                              generation_config = MODEL_CONFIG)

In [ ]:
# UPLOADING QUESTION PAPER

from google.colab import files
from pathlib import Path

# Upload the image file
uploaded = files.upload()

file_path = list(uploaded.keys())[0]  # Gets the first uploaded file name

import base64

def encode_pdf_to_base64(file_path):
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode("utf-8")

base64_pdf = encode_pdf_to_base64(file_path)
file_data = Path(file_path).read_bytes()

system_prompt = """
               You are a specialist in extracting and comprehending Handwritten/printed text from various file formats.
               Input files such as images, PDFs, and text documents will be provided,
               and your task is to return the extracted text in markdown format.
               You are NOT allowed to summarise the data, you must retrun all of it word to word.
               """

response = model.generate_content(
    contents=[
        {"role": "user", "parts": [  # Must be a list of dictionaries
            {"text": system_prompt},
            {"inline_data": {"mime_type": "application/pdf", "data": base64_pdf}}
        ]}
    ]
)

QuestionPaper = response.text
print(QuestionPaper)


In [ ]:
# UPLOADING ANSWER SHEET

from google.colab import files
from pathlib import Path

# Upload the image file
uploaded = files.upload()

file_path = list(uploaded.keys())[0]  # Gets the first uploaded file name

#for file_name in uploaded.keys():
#    print(f"Processing file: {file_name}")
#    file_path = Path(file_name)

import base64

def encode_pdf_to_base64(file_path):
    with open(file_path, "rb") as file:
        return base64.b64encode(file.read()).decode("utf-8")

base64_pdf = encode_pdf_to_base64(file_path)
file_data = Path(file_path).read_bytes()

system_prompt = """
               You are a specialist in extracting and comprehending Handwritten/printed text from various file formats.
               Input files such as images, PDFs, and text documents will be provided,
               and your task is to return the extracted text in markdown format.
               You are NOT allowed to summarise the data, you must retrun all of it word to word.
               """

response = model.generate_content(
    contents=[
        {"role": "user", "parts": [  # Must be a list of dictionaries
            {"text": system_prompt},
            {"inline_data": {"mime_type": "application/pdf", "data": base64_pdf}}
        ]}
    ]
)

AnswerPaper = response.text
print(AnswerPaper)

In [ ]:
# Grading prompt
strictness_level = 1.5 #out of 5

prompt_text = f"""
You are an AI educator with expertise in automated grading and feedback generation for student answers. You will be provided with two markdown files:
1. A **Question Paper** that includes:
   - All questions
   - The grading scheme for each question (mark distribution)
   - Any instructions or special rules (e.g., "Attempt any 4 out of 5 questions", "Internal choice", etc.)
2. A **Student Answer Sheet** that contains the student's answers in markdown format.
3. A **Strictness Level (1-5):**
   - **1 (Least Strict):** Accepts vague or incomplete answers and still gives decent marks.
   - **3 (Moderate Strictness):** Requires reasonable accuracy and inclusion of key concepts.
   - **5 (Most Strict):** Only highly accurate, well-structured answers receive full marks.

### Your tasks:

1. **Extract the Grading Scheme:**
   - Read the question paper and extract the grading scheme for each question.
   - Identify any instructions (such as internal choices) that affect how questions should be graded.

2. **Identify Relevant Keywords and Concepts:**
   - Without explicit keywords provided, analyze the question paper to determine which key terms or concepts are expected in the answer.
   - Use this understanding to search for those concepts in the student's answers.

3. **Map Questions to Answers:**
   - The order of answers in the answer sheet may be random. Carefully match each answer to its corresponding question from the question paper.
   - Ensure that each question’s answer is evaluated against the correct criteria.
   - The markdown text of answer sheet can have some incorrect text as it comes from an ocr, keep that in mind.
   - If the mcq answer is not correct but you see the option written (A/B/C/D) is correct then give correct marks.

4. **Grading Process (Based on Strictness Level):**
   - For each mapped question and answer pair, evaluate the answer based on the grading scheme and strictness level.
   - If the strictness level is **low (1-2):** minor errors, missing explanations, or vague reasoning may still get partial marks.
   - If the strictness level is **moderate (3):** requires answers to be mostly correct with reasonable completeness.
   - If the strictness level is **high (4-5):** strict evaluation, deducting marks for even minor mistakes or lack of clarity.
   - Provide a detailed explanation of how the marks were awarded.

5. **Feedback and Improvement Suggestions:**
   - Generate constructive feedback for each answer, highlighting strengths and areas for improvement.
   - Suggest topics or concepts that the student should review to improve their understanding.

6. **Final Summary:**
   - Sum up the marks from all questions to calculate the total grade.
   - Provide a comprehensive summary of the student's overall performance and list the topics they can focus on to improve.

### **Input Format:**
- **Question Paper:** Markdown file containing all questions, grading schemes, and instructions: {QuestionPaper}
- **Student Answer Sheet:** Markdown file containing the student's answers: {AnswerPaper}
- **Strictness Level:** A decimal value between 1 and 5 that determines how strictly the answers are graded: {strictness_level}

### **Output Format:**
Your response should include:
- A mapping of questions to answers.
- For each question:
  - The grade awarded (out of the specified marks).
  - An explanation of the grading.
  - Constructive feedback.
- A final summary with the total marks and overall improvement suggestions.

Please process the provided markdown inputs and generate a cleaned, coherent evaluation based on the specified strictness level.
"""

In [ ]:
from google import genai

client = genai.Client(api_key=api_key)
response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt_text
)
print(response.text)